In [31]:
from orion_recommend.datasets import DatasetMap, synthetic, utils
from orion.sources import S3Source
from orion.sources.io import read_csv, write_csv
from orion.sources import RedshiftSource
import pandas as pd

In [103]:
#source = RedshiftSource(query='SELECT * FROM publish.inventory_lookup WHERE productid IS NULL')
#df_stock = source.read_csv()

with S3Source(key="masters/uploads/customers/1560425511130_Peak_customers.csv", bucket="kilimanjaro-prod-datalake") as s3:
    df_cust = read_csv(s3)

df_trans = read_csv(S3Source(key="masters/uploads/transactions/1560426066385_Peak_transactions.csv", bucket="kilimanjaro-prod-datalake"))
df_prod = read_csv(S3Source(key="masters/uploads/product/1560425499995_Peak_product.csv", bucket="kilimanjaro-prod-datalake"))


/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [104]:
df_trans.drop(['orderdate', 'ordertime', 'shippingdate','grossprofit'], axis=1, inplace=True)

In [105]:
user_num = len(df_cust.customerkey.unique())
item_num = len(df_prod)

In [106]:
# 30pc of transactions have keys that are not in df_cust
pd.Series(df_trans.customerkey.unique()).isin(df_cust.customerkey).sum()/len(df_trans.customerkey.unique())
# 45pc of item numbers not in df_prod
pd.Series(df_trans.itemnumber.unique()).isin(df_prod.itemnumber).sum()/len(df_trans.itemnumber.unique())

0.5487752814984791

In [107]:
# Fix item number 
df_prod.itemnumber = pd.to_numeric(df_prod.itemnumber, errors='coerce') 

In [108]:
# Renumber customer keys
ints = [i for i in range(user_num)]
cust_keys = [i for i in df_cust.customerkey.unique()]
customer_key_dict = {i:j for i,j in zip( cust_keys, ints)}

In [109]:
# Drop nas
df_prod.dropna(subset=["itemnumber"], inplace=True)

In [110]:
#Renumber item ids
ints = [i for i in range(item_num)]
item_keys = [i for i in df_prod.itemnumber.unique()]
item_key_dict = {i:j for i,j in zip( item_keys, ints)}

In [111]:
# Remove keys not in customer and product tables
df_trans = df_trans.iloc[df_trans.customerkey[df_trans.customerkey.isin(df_cust.customerkey)].index,:]
df_trans.reset_index(inplace=True)
df_trans.drop(["index"], axis=1, inplace=True)
df_trans = df_trans.iloc[df_trans.itemnumber[df_trans.itemnumber.isin(df_prod.itemnumber)].index,:]

# Reset index before changing keys to integers
df_trans.reset_index(inplace=True)
df_trans.drop(["index"], axis=1, inplace=True)
df_prod.reset_index(inplace=True)
df_trans.drop(["index"], axis=1, inplace=True)
df_trans.drop(["index"], axis=1, inplace=True)


In [112]:
df_trans.itemnumber = [item_key_dict[df_trans.itemnumber[i]] for i in range(len(df_trans))]
df_prod.itemnumber = [item_key_dict[df_prod.itemnumber[i]] for i in range(len(df_prod))]
# Change keys to integers in dataframes
df_trans.customerkey = [customer_key_dict[df_trans.customerkey[i]] for i in range(len(df_trans))]
df_cust.customerkey = [customer_key_dict[df_cust.customerkey[i]] for i in range(len(df_cust))]

In [113]:
# Drop redundant cols. Fabric all unknown, item colour less info than colourvalue
df_prod.drop(["itemstylecode", "itemquarter", "itemcolour", "fabric"], axis=1, inplace=True)

In [114]:
# Drop columns
df_trans.drop(["index","salestransactionkey", "salesordernumber","discountpercent", "grosssales"], axis=1, inplace=True)

In [120]:
df = df_trans.merge(df_cust, on='customerkey', how = "outer")
df = df.merge(df_prod, on = "itemnumber")

In [121]:
df

,level_0,customerkey,shipcountry,postcode,itemnumber,pricetype,unitssold,loyaltyaccount,gender,index,brandcode,category,colourvalue,divisioncode,itemcategorycode,itemfamilycode,itemseason,productgroup
0,0.0,1536149,United Kingdom,S61,1876.0,Clearance,1.0,Yes,female,1876,NN,Mens,WHITE,FOOTWEAR,FM,FA,SS,Nylon Running
1,624.0,1318692,United Kingdom,HU5,1876.0,Clearance,1.0,No,male,1876,NN,Mens,WHITE,FOOTWEAR,FM,FA,SS,Nylon Running
2,645.0,513462,United Kingdom,BN1,1876.0,Clearance,1.0,No,male,1876,NN,Mens,WHITE,FOOTWEAR,FM,FA,SS,Nylon Running
3,6276448.0,513462,United Kingdom,BN1,1876.0,Clearance,1.0,No,male,1876,NN,Mens,WHITE,FOOTWEAR,FM,FA,SS,Nylon Running
4,7452604.0,513462,United Kingdom,BN1,1876.0,Clearance,1.0,No,male,1876,NN,Mens,WHITE,FOOTWEAR,FM,FA,SS,Nylon Running
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12648338,7584332.0,104646,United Kingdom,S12,138875.0,Full Price,1.0,Yes,female,138876,AA,Womens,BLACK,FOOTWEAR,FW,FA,SS,Basketball
12648339,7605707.0,104646,United Kingdom,S12,138875.0,Full Price,1.0,Yes,female,138876,AA,Womens,BLACK,FOOTWEAR,FW,FA,SS,Basketball
12648340,7605707.0,104646,United Kingdom,S12,138875.0,Full Price,1.0,Yes,female,138876,AA,Womens,BLACK,FOOTWEAR,FW,FA,SS,Basketball
12648341,7605707.0,104646,United Kingdom,S12,138875.0,Full Price,1.0,Yes,female,138876,AA,Womens,BLACK,FOOTWEAR,FW,FA,SS,Basketball


In [131]:
# Rename and drop columns
df.rename(columns={"customerkey":"user_id","itemnumber":"item_id","shipcountry":"country", "brandcode":"brand_id",
                  "colourvalue":"colour", }, inplace=True)


SyntaxError: invalid syntax (<ipython-input-131-9f33ed5d2141>, line 2)

In [133]:
df.drop(['level_0'], axis=1, inplace=True)

In [ ]:
df.gender.